In [1]:
#Importing  libraries

In [2]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')

from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from heapq import nlargest

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91818\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
with open(r"C:\Users\91818\Desktop\dailywork\Datasets_Day21-Natural Language Processing(NLP)\NLP-TM.txt",encoding='utf-8') as file:
    data=file.read()

In [12]:
#Text Cleansing

In [25]:
def remove_links(text):
    '''Takes a string and removes web links from it'''
    text = re.sub(r'http\S+', '', text) # remove http links
    text = re.sub(r'bit.ly/\S+', '', text) # rempve bitly links
    text = text.strip('[link]') # remove [links]
    return text

In [26]:
def remove_users(text):
    '''Takes a string and removes retweet and @user information'''
    text = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) # remove retweet
    text = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) # remove tweeted at
    return text

In [27]:
from nltk.corpus import stopwords
from string import punctuation
from heapq import nlargest

STOPWORDS = set(stopwords.words('english') + list(punctuation))
MIN_WORD_PROP, MAX_WORD_PROP = 0.1, 0.9

In [28]:
def clean_text(text, bigrams=False):
    text = remove_users(text)
    text = remove_links(text)
    text = text.lower() # lower case
    #tweet = re.sub('['+punctuation + ']+', ' ', tweet) # strip punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    text = re.sub('([0-9]+)', '', text) # remove numbers
    text_token_list = [word for word in text.split(' ')
                            if word not in STOPWORDS] # remove stopwords

   # tweet_token_list = [word_rooter(word) if '#' not in word else word
    #                    for word in tweet_token_list] # apply word rooter
    if bigrams:
        text_token_list = text_token_list+[text_token_list[i]+'_'+text_token_list[i+1]
                                            for i in range(len(text_token_list)-1)]
    tweet = ' '.join(text_token_list)
    return text

In [29]:
text=clean_text(data)

In [30]:
text

'in an article in cell, national institutes of health-funded researchers described how they used advanced genetic engineering techniques to transform a bacterial protein into a new research tool that may help monitor serotonin transmission with greater fidelity than current methods.preclinical experiments, primarily in mice, showed that the sensor could detect subtle, real-time changes in brain serotonin levels during sleep, fear, and social interactions, as well as test the effectiveness of new psychoactive drugs.the study was funded, in part, by the nih\'\'s brain research through advancing innovative neurotechnologies (brain) initiative which aims to revolutionize our understanding of the brain under healthy and disease conditions.the study was led by researchers in the lab of lin tian, phd, principal investigator at the university of california davis school of medicine. current methods can only detect broad changes in serotonin signaling. in this study, the researchers transformed 

In [ ]:
#Text Summarizer

In [31]:
def compute_word_frequencies(word_sentences):
    words = [word for sentence in word_sentences 
                     for word in sentence 
                         if word not in STOPWORDS]
    counter = Counter(words)
    limit = float(max(counter.values()))
    word_frequencies = {word: freq/limit 
                                for word,freq in counter.items()}
    # Drop words if too common or too uncommon
    word_frequencies = {word: freq 
                            for word,freq in word_frequencies.items() 
                                if freq > MIN_WORD_PROP 
                                and freq < MAX_WORD_PROP}
    return word_frequencies

In [32]:
def sentence_score(word_sentence, word_frequencies):
    return sum([ word_frequencies.get(word,0) 
                    for word in word_sentence])

In [33]:
def summarize(text:str, num_sentences=3):
    """
    Summarize the text, by return the most relevant sentences
     :text the text to summarize
     :num_sentences the number of sentences to return
    """
    text = text.lower() # Make the text lowercase
    
    sentences = sent_tokenize(text) # Break text into sentences 
    
    # Break sentences into words
    word_sentences = [word_tokenize(sentence) for sentence in sentences]
    
    # Compute the word frequencies
    word_frequencies = compute_word_frequencies(word_sentences)
    
    # Calculate the scores for each of the sentences
    scores = [sentence_score(word_sentence, word_frequencies) for word_sentence in word_sentences]
    sentence_scores = list(zip(sentences, scores))
    
    # Rank the sentences
    top_sentence_scores = nlargest(num_sentences, sentence_scores, key=lambda t: t[1])
    
    # Return the top sentences
    return [t[0] for t in top_sentence_scores]

In [34]:
len(sent_tokenize(text))



1791

In [35]:
from nltk.corpus import stopwords
from string import punctuation
from heapq import nlargest

STOPWORDS = set(stopwords.words('english') + list(punctuation))
MIN_WORD_PROP, MAX_WORD_PROP = 0.1, 0.9

In [36]:
summarize(text, num_sentences=3)

['example technologies might include crispr-based methods, high-throughput use of compact (adeno-associated virus (aav)-sized) enhancers that can control hundreds or thousands of specific cell types; monoclonal antibodies and/or nanobodies against cell type-specific surface proteins for pseudotyping lentiviruses; aav serotypes with novel cell specificities; permanent, activity-dependent cell-marking methods; and methods that combine approaches and targets (e.g., split-gal with two enhancers, split-gal with pseudotyped lentivirus).reversible, cell type-based manipulation of brain activity would advance understanding of fundamental principles of brain function, but also guide novel therapies for brain disorders through the use of animal models.',
 "they hope that it will help researchers gain a better understanding of the critical role serotonin plays in our daily lives and in many psychiatric conditions.if the us national institutes of health has its way, there would be some extensive r

In [37]:
#Topic MOdeling-LDA

In [38]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

#raw_text = "This is the first sentence. This is the second sentence. This is the third sentence."

sentences = sent_tokenize(text)
#print(sentences)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91818\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
#pip install spacy
from spacy.lang.en.stop_words import STOP_WORDS as stopwords


tfidf_para_vectorizer = TfidfVectorizer(stop_words = stopwords, min_df = 5,max_df = 0.7)
tfidf_para_vectors = tfidf_para_vectorizer.fit_transform(sentences)
tfidf_para_vectors.shape

from sklearn.feature_extraction.text import CountVectorizer

count_para_vectorizer = CountVectorizer(stop_words = stopwords, min_df = 5, max_df = 0.7)
count_para_vectors = count_para_vectorizer.fit_transform(sentences)

from sklearn.decomposition import LatentDirichletAllocation

lda_para_model = LatentDirichletAllocation(n_components = 10, random_state = 42)
W_lda_para_matrix = lda_para_model.fit_transform(count_para_vectors)
H_lda_para_matrix = lda_para_model.components_

def display_topics(model, features, no_top_words = 5):
    for topic, word_vector in enumerate(model.components_):
        total = word_vector.sum()
        largest = word_vector.argsort()[::-1] # invert sort order
        print("\nTopic %02d" % topic)
        for i in range(0, no_top_words):
            print("  %s (%2.2f)" % (features[largest[i]],
                  word_vector[largest[i]]*100.0/total))

display_topics(lda_para_model, tfidf_para_vectorizer.get_feature_names())

C:\Users\91818\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(



Topic 00
  brain (7.42)
  nih (6.34)
  initiative (5.37)
  funded (2.40)
  funding (2.30)

Topic 01
  cell (8.36)
  types (4.22)
  brain (3.56)
  activity (1.72)
  type (1.68)

Topic 02
  human (2.14)
  approaches (2.01)
  include (1.94)
  methods (1.73)
  imaging (1.70)

Topic 03
  technologies (3.09)
  humans (2.88)
  tools (2.61)
  use (2.55)
  methods (2.17)

Topic 04
  data (3.94)
  brain (1.98)
  high (1.40)
  methods (1.37)
  goal (1.25)

Topic 05
  time (2.15)
  studies (1.67)
  real (1.38)
  signals (1.21)
  mice (1.18)

Topic 06
  data (5.35)
  brain (4.77)
  initiative (1.51)
  new (1.36)
  tools (1.28)

Topic 07
  research (4.29)
  neuroscience (2.79)
  training (2.35)
  quantitative (1.51)
  human (1.49)

Topic 08
  brain (7.81)
  goals (2.62)
  term (2.26)
  opportunities (1.57)
  data (1.42)

Topic 09
  brain (7.82)
  area (2.48)
  function (2.14)
  priority (2.13)
  disorders (1.72)


C:\Users\91818\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [41]:
#Named Entity Recognition

In [42]:
#!python -m spacy download en_core_web_sm


In [43]:
import spacy
from spacy import displacy

# Load the pre-trained model
nlp = spacy.load("en_core_web_sm")

# Process the text
#raw_text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text[:1000])

# Extract entities
          
displacy.render(doc, style='ent', jupyter=True)


In [44]:
displacy.render(doc, style='dep')

In [45]:
#POS Tagging

In [46]:
#pos tagging for fisrt 100 words
for token in doc[:100]:
    print(token.text, token.pos_)

in ADP
an DET
article NOUN
in ADP
cell NOUN
, PUNCT
national ADJ
institutes NOUN
of ADP
health NOUN
- PUNCT
funded VERB
researchers NOUN
described VERB
how SCONJ
they PRON
used VERB
advanced ADJ
genetic ADJ
engineering NOUN
techniques NOUN
to PART
transform VERB
a DET
bacterial ADJ
protein NOUN
into ADP
a DET
new ADJ
research NOUN
tool NOUN
that PRON
may AUX
help VERB
monitor VERB
serotonin ADJ
transmission NOUN
with ADP
greater ADJ
fidelity NOUN
than ADP
current ADJ
methods.preclinical ADJ
experiments NOUN
, PUNCT
primarily ADV
in ADP
mice NOUN
, PUNCT
showed VERB
that SCONJ
the DET
sensor NOUN
could AUX
detect VERB
subtle ADJ
, PUNCT
real ADJ
- PUNCT
time NOUN
changes NOUN
in ADP
brain NOUN
serotonin NOUN
levels NOUN
during ADP
sleep NOUN
, PUNCT
fear NOUN
, PUNCT
and CCONJ
social ADJ
interactions NOUN
, PUNCT
as ADV
well ADV
as ADP
test VERB
the DET
effectiveness NOUN
of ADP
new ADJ
psychoactive ADJ
drugs.the DET
study NOUN
was AUX
funded VERB
, PUNCT
in ADP
part NOUN
, PUNCT
by ADP